### Name : V Yaswanth Krishna --                           Roll No. : 17BT30025
### Name : Juluri Shree Shiva Teja --                    Roll No. : 17EE10015
### Name : Murali Karthik --                                     Roll No. : 17EE10012
### Name : Sai Ram Tadiboyina --                           Roll No. : 17AG36017


## PART1 : Sentence classification

In [3]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [4]:
df_train = pd.read_csv("traindata.csv")
df_train.head(5)

,category,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,science,Outer space is not friendly to life. Extreme t...,NaN,NaN,NaN,NaN
1,sports,"Tennis, original name lawn tennis, game in whi...",NaN,NaN,NaN,NaN
2,business,One woman who frequently flew on Southwest was...,NaN,NaN,NaN,NaN
3,covid,"In December 2019, almost seven years after the...",NaN,NaN,NaN,NaN
4,science,Any life-forms that somehow find themselves in...,NaN,NaN,NaN,NaN


In [5]:
#splitting the sentences into words and making them to lowercase
df_train["text"]= df_train["text"].str.lower().str.split()
df_train.head()

,category,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,science,"[outer, space, is, not, friendly, to, life., e...",NaN,NaN,NaN,NaN
1,sports,"[tennis,, original, name, lawn, tennis,, game,...",NaN,NaN,NaN,NaN
2,business,"[one, woman, who, frequently, flew, on, southw...",NaN,NaN,NaN,NaN
3,covid,"[in, december, 2019,, almost, seven, years, af...",NaN,NaN,NaN,NaN
4,science,"[any, life-forms, that, somehow, find, themsel...",NaN,NaN,NaN,NaN


In [6]:
#removing the stopwords from the words present in the data
df_train['text']=df_train['text'].apply(lambda x: [word for word in x if word not in stop_words])
df_train.head()

,category,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,science,"[outer, space, friendly, life., extreme, tempe...",NaN,NaN,NaN,NaN
1,sports,"[tennis,, original, name, lawn, tennis,, game,...",NaN,NaN,NaN,NaN
2,business,"[one, woman, frequently, flew, southwest, cons...",NaN,NaN,NaN,NaN
3,covid,"[december, 2019,, almost, seven, years, mers, ...",NaN,NaN,NaN,NaN
4,science,"[life-forms, somehow, find, void, soon, die., ...",NaN,NaN,NaN,NaN


In [7]:
#joining sentences to count the freq of words
df_train["text"]=df_train["text"].str.join(" ")
df_train.head()

,category,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,science,outer space friendly life. extreme temperature...,NaN,NaN,NaN,NaN
1,sports,"tennis, original name lawn tennis, game two op...",NaN,NaN,NaN,NaN
2,business,one woman frequently flew southwest constantly...,NaN,NaN,NaN,NaN
3,covid,"december 2019, almost seven years mers 2012 ou...",NaN,NaN,NaN,NaN
4,science,life-forms somehow find void soon die. unless ...,NaN,NaN,NaN,NaN


In [8]:
#freq of occurence of words  with out stop words
class_cond_dist = pd.DataFrame(df_train.text.str.split(expand=True).stack().value_counts())
class_cond_dist.reset_index(level=0, inplace=True)
class_cond_dist.columns = ['words', 'count']
class_cond_dist

,words,count
0,tennis,12
1,customer,11
2,service,10
3,employees,9
4,customers,9
...,...,...
901,flew,1
902,period,1
903,january,1
904,examples,1


In [9]:
#Calculating the prior distribution of the labels:

pd_labels = pd.DataFrame(df_train.category.str.split(expand=True).stack().value_counts())
pd_labels.reset_index(level=0, inplace=True)
pd_labels.columns = ['labels', 'count']

prob=[]
for i in range(4):
    prob.append(pd_labels["count"][i]/80)
pd_labels["prob"]=prob
pd_labels

,labels,count,prob
0,covid,21,0.2625
1,sports,20,0.2500
2,science,20,0.2500
3,business,19,0.2375


In [10]:
a = df_train.sort_values(by ='category', ascending = 1) 
label={'business':a[:19],'covid':a[19:40],'science':a[40:60],'sports':a[60:]}
vocab={}
for cat in label.keys():
    label[cat]["text"]=label[cat]["text"].str.lower().str.split()  #splitting the sentences 
    label[cat]["text"]=label[cat]["text"].apply(lambda x: [word for word in x if word not in stop_words]) #removing stop wprds
    label[cat]["text"]=label[cat]["text"].str.join(" ") # joining the words after removing stopwords
    
    #calculating the freq of words in respective labels and building vocabulary:
    
    vocab[cat]=pd.DataFrame(label[cat].text.str.split(expand=True).stack().value_counts())
    vocab[cat].reset_index(level=0, inplace=True)
    vocab[cat].columns = ['word', 'count']
    vocab[cat]=vocab[cat][vocab[cat]['count'] >= 0]

<ipython-input-10-0780cfda61bd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label[cat]["text"]=label[cat]["text"].str.lower().str.split()  #splitting the sentences
<ipython-input-10-0780cfda61bd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label[cat]["text"]=label[cat]["text"].apply(lambda x: [word for word in x if word not in stop_words]) #removing stop wprds
<ipython-input-10-0780cfda61bd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [11]:
#finding class conditional probabilities

prob={}
for cat in label.keys():   
    prob[cat]=np.zeros(len(class_cond_dist)) 
    
    for i in range(len(vocab[cat])):
        for j in range(len(class_cond_dist)):
            
            if class_cond_dist['words'][j] == vocab[cat]['word'][i] :
                
                prob[cat][j]=vocab[cat]['count'][i]/class_cond_dist['count'][j]
                
                if prob[cat][j] == 1:
                    prob[cat][j] = 0.999
                    
    for i in range(len(class_cond_dist)):
        if prob[cat][i] == 0:
            prob[cat][i] =0.001
            
    class_cond_dist= pd.concat([class_cond_dist, pd.Series(prob[cat], index=class_cond_dist.index, name='P(word|label='+cat+')')], axis=1)
class_cond_dist

,words,count,P(word|label=business),P(word|label=covid),P(word|label=science),P(word|label=sports)
0,tennis,12,0.001,0.001,0.001,0.999
1,customer,11,0.999,0.001,0.001,0.001
2,service,10,0.300,0.001,0.001,0.700
3,employees,9,0.999,0.001,0.001,0.001
4,customers,9,0.999,0.001,0.001,0.001
...,...,...,...,...,...,...
901,flew,1,0.999,0.001,0.001,0.001
902,period,1,0.001,0.001,0.001,0.999
903,january,1,0.001,0.999,0.001,0.001
904,examples,1,0.999,0.001,0.001,0.001


In [12]:
df_test = pd.read_csv("testdata.csv")
df_test["text"]= df_test["text"].str.lower().str.split()
df_test['text']=df_test['text'].apply(lambda x: [word for word in x if word not in stop_words])
df_test.head()

,category,text
0,science,"[estimates, 1,000-micrometer, pellets, could, ..."
1,science,"[“that’s, enough, time, potentially, get, mars..."
2,science,"[exactly, clumps, microbes, might, get, expell..."
3,science,"[microbes, might, get, kicked, small, meteorit..."
4,science,"[someday,, microbial, life, ever, discovered, ..."


In [13]:
#estimating the probabilities of every text into labels:

prob_test={}
for cat in label.keys(): 
    prob_test[cat]=np.zeros(len(df_test))
    for i in range(len(df_test)):
        prob_test[cat][i]=1
        for j in range(len(df_test['text'][i])):
            for k in range(len(class_cond_dist)):
                if df_test['text'][i][j] == class_cond_dist["words"][k]:
                    prob_test[cat][i]=prob_test[cat][i]*class_cond_dist['P(word|label='+cat+')'][k]
        prob_test[cat][i]=prob_test[cat][i]*pd_labels['prob'][0]

    df_test = pd.concat([df_test, pd.Series(prob_test[cat], index=df_test.index, name=cat)], axis=1)

#predicting the class of the given label based on prob values:

df_test["Predicted"]=df_test[["business","covid","science","sports"]].idxmax(axis=1)
    
df_test

,category,text,business,covid,science,sports,Predicted
0,science,"[estimates, 1,000-micrometer, pellets, could, ...",2.625000e-16,1.750000e-13,8.715052e-02,2.625000e-16,science
1,science,"[“that’s, enough, time, potentially, get, mars...",1.748250e-16,8.750000e-20,2.614516e-10,2.622375e-19,science
2,science,"[exactly, clumps, microbes, might, get, expell...",2.622375e-13,2.625000e-16,2.614516e-04,2.625000e-16,science
3,science,"[microbes, might, get, kicked, small, meteorit...",2.622375e-25,2.625000e-28,2.604073e-04,2.625000e-28,science
4,science,"[someday,, microbial, life, ever, discovered, ...",2.625000e-07,2.625000e-07,2.619753e-01,2.625000e-07,science
5,sports,"[tennis,, service, correctly, returned,, playe...",8.859375e-33,1.312500e-38,1.312500e-38,1.365771e-05,sports
6,sports,"[may, occur, tennis, player, fails, hit, ball,...",6.562500e-47,6.562500e-47,8.750000e-47,9.716546e-02,sports
7,sports,"[win, game,, tennis, player, must, win, four, ...",2.625000e-25,2.625000e-25,2.622375e-22,2.606680e-04,sports
8,sports,"[tennis,, never, satisfactorily, explained, th...",2.625000e-16,2.625000e-16,2.622375e-13,2.614516e-04,sports
9,sports,"[tennis,, server’s, score, called, first;, thu...",2.910836e-20,2.625000e-31,2.625000e-31,1.159684e-07,sports


## PART2 : Sentence Completion

In [15]:
#Loading the data and splitting it into words

df = pd.read_csv("40.csv")
vocab=[]

for i in range(df.shape[0]):
    vocab.append(df['text'][i].split())
print(vocab[0:5])    
#print(len(vocab))    

[['In', 'the', 'midst', 'of', 'the', 'COVID-19', 'pandemic,', 'eating', 'healthy', 'food', 'remains', 'an', 'important', 'part', 'of', 'maintaining', 'your', 'health.'], ['While', 'there', 'are', 'no', 'specific', 'foods', 'that', 'can', 'help', 'protect', 'you', 'from', 'the', 'virus,', 'a', 'nutritious', 'diet', 'can', 'boost', 'your', 'immune', 'system', 'or', 'help', 'you', 'fight', 'off', 'symptoms.'], ['You', 'may', 'not', 'be', 'able', 'to', 'share', 'meals', 'with', 'friends', 'and', 'loved', 'ones,', 'but', 'there', 'are', 'lots', 'of', 'other', 'ways', 'to', 'eat', 'well', 'and', 'support', 'your', 'health', 'at', 'this', 'difficult', 'time.'], ['Eating', 'a', 'healthy', 'diet', 'is', 'not', 'about', 'strict', 'limitations,', 'staying', 'unrealistically', 'thin,', 'or', 'depriving', 'yourself', 'of', 'the', 'foods', 'you', 'love.', 'Rather,', 'it’s', 'about', 'feeling', 'great,', 'having', 'more', 'energy,', 'improving', 'your', 'health,', 'and', 'boosting', 'your', 'mood.'],

In [16]:
#Removing the punctuation marks in the sentences

temp=str.maketrans('','',string.punctuation)
stripped_vocab=[]
for sentence in vocab:
    stripped_vocab.append([word.translate(temp).lower() for word in sentence])
print(stripped_vocab[0:5])

#print(len(stripped_vocab))

[['in', 'the', 'midst', 'of', 'the', 'covid19', 'pandemic', 'eating', 'healthy', 'food', 'remains', 'an', 'important', 'part', 'of', 'maintaining', 'your', 'health'], ['while', 'there', 'are', 'no', 'specific', 'foods', 'that', 'can', 'help', 'protect', 'you', 'from', 'the', 'virus', 'a', 'nutritious', 'diet', 'can', 'boost', 'your', 'immune', 'system', 'or', 'help', 'you', 'fight', 'off', 'symptoms'], ['you', 'may', 'not', 'be', 'able', 'to', 'share', 'meals', 'with', 'friends', 'and', 'loved', 'ones', 'but', 'there', 'are', 'lots', 'of', 'other', 'ways', 'to', 'eat', 'well', 'and', 'support', 'your', 'health', 'at', 'this', 'difficult', 'time'], ['eating', 'a', 'healthy', 'diet', 'is', 'not', 'about', 'strict', 'limitations', 'staying', 'unrealistically', 'thin', 'or', 'depriving', 'yourself', 'of', 'the', 'foods', 'you', 'love', 'rather', 'it’s', 'about', 'feeling', 'great', 'having', 'more', 'energy', 'improving', 'your', 'health', 'and', 'boosting', 'your', 'mood'], ['healthy', 'e

In [17]:
#Removing stop words from sentences

stopWords=stopwords.words('english')

filtered_vocab=[]
for sentence in stripped_vocab:
    filtered_vocab.append([word for word in sentence if word not in stopWords])
print(filtered_vocab[0:5])        
#print(len(filtered_vocab))

[['midst', 'covid19', 'pandemic', 'eating', 'healthy', 'food', 'remains', 'important', 'part', 'maintaining', 'health'], ['specific', 'foods', 'help', 'protect', 'virus', 'nutritious', 'diet', 'boost', 'immune', 'system', 'help', 'fight', 'symptoms'], ['may', 'able', 'share', 'meals', 'friends', 'loved', 'ones', 'lots', 'ways', 'eat', 'well', 'support', 'health', 'difficult', 'time'], ['eating', 'healthy', 'diet', 'strict', 'limitations', 'staying', 'unrealistically', 'thin', 'depriving', 'foods', 'love', 'rather', 'it’s', 'feeling', 'great', 'energy', 'improving', 'health', 'boosting', 'mood'], ['healthy', 'eating', 'doesn’t', 'overly', 'complicated', 'feel', 'overwhelmed', 'conflicting', 'nutrition', 'diet', 'advice', 'you’re', 'alone']]


In [18]:
#As it observed that some words of type 'A—B' are not splitted into A and B, they are splitted manually

vocabulary=[]
for sentence in filtered_vocab:
    sent=[]
    for word in sentence:
        sent+=word.split('—')
    vocabulary.append(sent)
print(vocabulary[0:5])

#print(len(vocabulary))

[['midst', 'covid19', 'pandemic', 'eating', 'healthy', 'food', 'remains', 'important', 'part', 'maintaining', 'health'], ['specific', 'foods', 'help', 'protect', 'virus', 'nutritious', 'diet', 'boost', 'immune', 'system', 'help', 'fight', 'symptoms'], ['may', 'able', 'share', 'meals', 'friends', 'loved', 'ones', 'lots', 'ways', 'eat', 'well', 'support', 'health', 'difficult', 'time'], ['eating', 'healthy', 'diet', 'strict', 'limitations', 'staying', 'unrealistically', 'thin', 'depriving', 'foods', 'love', 'rather', 'it’s', 'feeling', 'great', 'energy', 'improving', 'health', 'boosting', 'mood'], ['healthy', 'eating', 'doesn’t', 'overly', 'complicated', 'feel', 'overwhelmed', 'conflicting', 'nutrition', 'diet', 'advice', 'you’re', 'alone']]


In [19]:
#Estimating prior probabilities

prior_prob={}
total_words_count=0
sum=0
for sentence in vocabulary:
    for word in sentence:
        if word in prior_prob.keys():
            prior_prob[word]+=1
            total_words_count+=1
        else: 
            prior_prob[word]=1
            total_words_count+=1
for key in prior_prob.keys():
    prior_prob[key]/=total_words_count
    sum+=prior_prob[key]

#print(prior_prob)    
print("Total number of words in vocabulary =",total_words_count)
print("Total number of distinct words in vocabulary =",len(prior_prob))
print("Prior probabilities sum =",sum)

#print(len(prior_prob.keys()))

Total number of words in vocabulary = 587
Total number of distinct words in vocabulary = 382
Prior probabilities sum = 1.0000000000000007


In [20]:
#Creating dataframe for posterior probabilities

a = np.zeros(shape=(len(prior_prob),len(prior_prob)))
posterior_prob = pd.DataFrame(a,columns=list(prior_prob.keys()))
posterior_prob['y']=list(prior_prob.keys())
posterior_prob.set_index('y',inplace=True)

#posterior_prob

In [21]:
#Calculating posterior probabilities
#p[word][y] denotes the P(word|y)=Probability of word given y

for y in list(prior_prob.keys()):
    sentence_count=0
 
    for sentence in vocabulary:
        if y in sentence:
            sentence_count+=1
            for word in set(sentence):
                posterior_prob[word][y]+=1
    for word in list(prior_prob.keys()):
        posterior_prob[word][y]=posterior_prob[word][y]/sentence_count

#Replacing 0 with 1e-8        

posterior_prob=posterior_prob.replace(0,1e-8)    
posterior_prob    

,midst,covid19,pandemic,eating,healthy,food,remains,important,part,maintaining,...,onion,plain,salads,steamed,veggies,quickly,bland,taste,vegetable,dishes
y,,,,,,,,,,,,,,,,,,,,,
midst,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
covid19,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
pandemic,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
eating,8.333333e-02,8.333333e-02,8.333333e-02,1.000000e+00,3.333333e-01,3.333333e-01,8.333333e-02,8.333333e-02,8.333333e-02,8.333333e-02,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
healthy,7.142857e-02,7.142857e-02,7.142857e-02,2.857143e-01,1.000000e+00,1.428571e-01,7.142857e-02,2.142857e-01,7.142857e-02,7.142857e-02,...,7.142857e-02,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
quickly,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
bland,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
taste,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [22]:
#Loading the test dataset and refining the text with split, removing punctuation,removing stop words

df = pd.read_csv("10.csv")
test_vocab=[]
for i in range(df.shape[0]):
    test_vocab.append(df['text'][i].split())
print(test_vocab)    
#print(len(test_vocab))    



temp=str.maketrans('','',string.punctuation)
stripped_test_vocab=[]
for sentence in test_vocab:
    stripped_test_vocab.append([word.translate(temp).lower() for word in sentence])
print(stripped_test_vocab)

#print(len(stripped_test_vocab))


filtered_test_vocab=[]
for sentence in stripped_test_vocab:
    filtered_test_vocab.append([word for word in sentence if word not in stopWords+['']])
print(filtered_test_vocab)        
#print(len(filtered_test_vocab))


test_vocabulary=filtered_test_vocab

[['Eating', 'healthy', 'food', 'is', 'important', 'for', 'maintaining', 'good', '_____'], ['Following', 'a', 'healthy', 'diet', 'will', 'boost', 'your', '_____'], ['Avoid', 'eating', 'chemical', 'additives,', 'added', 'sugars', 'in', 'your', 'diet.', 'Switch', 'to', 'a', 'healthy', '____'], ['Your', 'diet', 'should', 'be', 'rich', 'of', 'vitamins', 'D,K,', 'calcium', 'and', '______'], ['The', 'healthier', 'the', 'food', 'you', 'eat,', 'the', 'better', 'you’ll', 'feel', 'after', 'a', '_____.'], ['Dehydration', 'causes', 'tiredness,', 'low', 'energy', 'and', 'head', 'aches.', 'Drink', 'plenty', 'of', '______'], ['People', 'with', 'kidney', 'disease', 'should', 'avoid', 'eating', 'high', 'amounts', 'of', '______'], ['We', 'should', 'avoid', 'eating', 'transfats.', 'Eating', 'healthy', 'fats', 'and', 'dietary', 'fibre', 'can', 'help', 'us', 'lose', '____'], ['Mindless', 'eating', 'is', 'often', 'caused', 'by', 'eating', 'alone.', 'We', 'should', 'avoid', 'eating', 'while', 'we', 'are', 'in

In [23]:
#creating dataframe for posterior probabilities on test dataset

a = np.zeros(shape=(len(prior_prob),len(test_vocabulary)))
test_posterior_prob = pd.DataFrame(a,columns=[i for i in range(len(test_vocabulary))])
test_posterior_prob['y']=list(prior_prob.keys())
test_posterior_prob.set_index('y',inplace=True)

#test_posterior_prob

In [24]:
#calculating posterior probabilities on test dataset and predicting the last word along with their probability 

for i in range(len(test_vocabulary)):
    sentence=test_vocabulary[i]
    for y in list(prior_prob.keys()):
        test_posterior_prob[i][y]=prior_prob[y]
        for word in set(sentence):
            if word in list(prior_prob.keys()):
                test_posterior_prob[i][y]*=posterior_prob[word][y]
    col_sum=test_posterior_prob[i].sum()    
    test_posterior_prob[i]/=col_sum
    print(df['text'][i],test_posterior_prob[i].idxmax()," with probability",test_posterior_prob[i].max())
test_posterior_prob            

Eating healthy food is important for maintaining good _____ midst  with probability 0.1589296951841983
Following a healthy diet will boost your _____ diet  with probability 0.18335329356299687
Avoid eating chemical additives, added sugars in your diet. Switch to a healthy ____ feeling  with probability 0.9651884585810944
Your diet should be rich of vitamins D,K, calcium and ______ calcium  with probability 0.11492990552613866
The healthier the food you eat, the better you’ll feel after a _____. better  with probability 0.2606071168546024
Dehydration causes tiredness, low energy and head aches. Drink plenty of ______ water  with probability 0.1299761183925986
People with kidney disease should avoid eating high amounts of ______ disease  with probability 0.9999999785253613
We should avoid eating transfats. Eating healthy fats and dietary fibre can help us lose ____ foods  with probability 0.9999999999997894
Mindless eating is often caused by eating alone. We should avoid eating while we 

,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
midst,1.589297e-01,1.389040e-17,6.177206e-39,7.661994e-26,2.084857e-40,6.498806e-42,8.000000e-32,1.195742e-43,1.903235e-41,4.095999e-30
covid19,1.589297e-01,1.389040e-17,6.177206e-39,7.661994e-26,2.084857e-40,6.498806e-42,8.000000e-32,1.195742e-43,1.903235e-41,4.095999e-30
pandemic,1.589297e-01,1.389040e-17,6.177206e-39,7.661994e-26,2.084857e-40,6.498806e-42,8.000000e-32,1.195742e-43,1.903235e-41,4.095999e-30
eating,1.471571e-03,9.260267e-10,4.118137e-31,1.532399e-17,4.826058e-19,6.498806e-34,6.666667e-17,1.383961e-14,1.835682e-05,1.365333e-22
healthy,1.390056e-03,5.953029e-02,1.058950e-30,3.283712e-10,2.978367e-33,6.498806e-34,2.285714e-24,7.320871e-29,3.884153e-27,1.194169e-16
...,...,...,...,...,...,...,...,...,...,...
quickly,1.589297e-41,1.389040e-25,6.177206e-55,7.661994e-26,2.084857e-48,6.498806e-34,8.000000e-40,1.195742e-59,1.903235e-49,4.095999e-46
bland,1.589297e-41,1.389040e-25,6.177206e-55,7.661994e-26,2.084857e-48,6.498806e-34,8.000000e-40,1.195742e-59,1.903235e-49,4.095999e-46
taste,1.589297e-41,1.389040e-25,6.177206e-55,7.661994e-26,2.084857e-48,6.498806e-34,8.000000e-40,1.195742e-59,1.903235e-49,4.095999e-46
